### Mise en situation
Vous êtes intégré à une nouvelle équipe de chercheurs de la Food and Agriculture Organization of the United Nations (FAO), l'un des organes qui compose l'ONU et dont l'objectif est d' « aider à construire un monde libéré de la faim ».

Votre équipe est chargée de réaliser une étude de grande ampleur sur le thème de la sous-nutrition dans le monde.

Le problème de la faim est complexe et peut avoir de multiples causes, différentes selon les pays. L’étape préliminaire de cette étude sera donc d’établir un “état de l’art” des recherches déjà publiées, mais également de mener une étude statistique destinée à orienter les recherches vers des pays particuliers, et de mettre en lumière différentes causes de la faim. Ainsi, une poignée de data analysts (dont vous !) a été sélectionnée pour mener cette étape préliminaire. Lors de la première réunion, vous avez été désigné pour mettre une place la base de données que votre équipe pourra requêter (en SQL) à souhait pour réaliser cette étude statistique.

### Les données
Les données sont constituées de 5 fichiers:

- fr_animaux.csv : multiples indicateurs de production des produits animaux en 2013
- fr_population.csv: population mondiale par pays en 2013
- fr_vegetaux.csv: multiples indicateurs de production des produits végétaux en 2013
- fr_céréales.csv: quantité de céréales produites au niveau mondial en 2013    
- fr_sousalimentation.csv: nombre de personnes sous alimentées dans le monde de 2013 à 2017.

Créez un dataframe contenant les informations de population de chaque pays. Calculez le nombre total d’humains sur la planète. Critiquez votre résultat. En cas d’anomalie, analysez et effectuer les corrections nécessaires.

## Préparation de mes données

In [1]:
# j'importe les differentes librairies pour m'aider à analyser mes dataframes
import pandas as pd
import numpy as np

In [2]:
# j'importe mes differentes dataframes pour mon projet
df_population = pd.read_csv('fr_population.csv')
df_cereales = pd.read_csv('fr_cereales.csv')
df_sousalimentation = pd.read_csv('fr_sousalimentation.csv')
df_animal = pd.read_csv('fr_animaux.csv')                               
df_vegetaux = pd.read_csv('fr_vegetaux.csv')

In [3]:
# Je filtre mes colonnes qui ne seront pas vraiment utilisé dans mon analyse
df_population.columns = df_population.columns = df_population.columns.str.strip().str.lower().str.replace(' ', '_')
df_population.head()

,code_domaine,domaine,code_zone,zone,code_élément,élément,code_produit,produit,code_année,année,unité,valeur,symbole,description_du_symbole
0,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,2,Afghanistan,511,Population totale,2501,Population,2013,2013,1000 personnes,30552,NaN,Donnée officielle
1,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,202,Afrique du Sud,511,Population totale,2501,Population,2013,2013,1000 personnes,52776,NaN,Donnée officielle
2,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,3,Albanie,511,Population totale,2501,Population,2013,2013,1000 personnes,3173,NaN,Donnée officielle
3,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,4,Algérie,511,Population totale,2501,Population,2013,2013,1000 personnes,39208,NaN,Donnée officielle
4,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,79,Allemagne,511,Population totale,2501,Population,2013,2013,1000 personnes,82727,NaN,Donnée officielle


## Question 1 : donnez le résultat de votre calcul pour l'année 2013.

In [4]:
# Je renomme mes colonnes
df_population.rename(columns={'zone': 'pays', 'valeur': 'population', 'description_du_symbole': 'symbol'}, inplace=True)
df_population_q1 = df_population[['code_zone', 'pays', 'population', 'symbol']]
df_population.head(2)

,code_domaine,domaine,code_zone,pays,code_élément,élément,code_produit,produit,code_année,année,unité,population,symbole,symbol
0,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,2,Afghanistan,511,Population totale,2501,Population,2013,2013,1000 personnes,30552,NaN,Donnée officielle
1,FBSH,Bilans Alimentaire (Ancienne méthodologie et p...,202,Afrique du Sud,511,Population totale,2501,Population,2013,2013,1000 personnes,52776,NaN,Donnée officielle


In [5]:
# Je visualise mon dataframe et je constate que ma ligne 'max' est très élevé
df_population_q1.population.describe()

count    1.750000e+02
mean     4.807996e+04
std      1.786327e+05
min      5.400000e+01
25%      2.543500e+03
50%      9.413000e+03
75%      2.888150e+04
max      1.416667e+06
Name: population, dtype: float64

In [6]:
# Je classe par ordre décroissant la colonne 'population' et je constate que la chine est compter 2 fois
df_population_q1.sort_values(by='population', ascending=False).head()

,code_zone,pays,population,symbol
33,351,Chine,1416667,"Agrégat, peut inclure des données officielles,..."
36,41,"Chine, continentale",1385567,Donnée officielle
75,100,Inde,1252140,Donnée officielle
55,231,États-Unis d'Amérique,320051,Donnée officielle
76,101,Indonésie,249866,Donnée officielle


In [7]:
# je vais supprimer cette ligne
df_population_q1.drop([33], inplace=True)
world_pop = sum(df_population_q1['population']) * 1000
print("Le total de la population dans le monde s'élève à {0:,}".format(world_pop))

Le total de la population dans le monde s'élève à 6,997,326,000


C:\Users\lilop\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## Question 2 : Identifiez les redondances

In [8]:
# je dois selectionner toutes les lignes éléments pour pouvoir savoir à quoi correspondent les valeurs de mes variables
# je filtre mon dataframe comme pour l'exercice précedent
df_vegetaux.columns = df_vegetaux.columns = df_vegetaux.columns.str.strip().str.lower().str.replace(' ', '_')
df_vegetaux.rename(columns={'zone': 'pays', 'valeur': 'quantite', 'description_du_symbole': 'symbol'}, inplace=True)
df_vegetaux_q2 = df_vegetaux[['code_zone', 'pays', 'élément', 'produit', 'code_produit', 'unité', 'quantite', 'symbol']]

# Suppression des valeurs en double(la Chine) via son index
df_vegetaux_q2.drop(df_vegetaux_q2[df_vegetaux_q2['code_zone'] == 351].index, inplace=True)

In [9]:
# je vais selectionner toutes les lignes concernant le blé en France pour pouvoir calculer mes redondances
df_vegetaux_q2_france = df_vegetaux_q2[(df_vegetaux_q2.pays == 'France') & (df_vegetaux_q2.code_produit == 2511)]
df_vegetaux_q2_france

,code_zone,pays,élément,produit,code_produit,unité,quantite,symbol
37487,68,France,Production,Blé,2511,Milliers de tonnes,38614.00,Données standardisées
37488,68,France,Importations - Quantité,Blé,2511,Milliers de tonnes,2055.00,Données standardisées
37489,68,France,Variation de stock,Blé,2511,Milliers de tonnes,1131.00,Données standardisées
37490,68,France,Exportations - Quantité,Blé,2511,Milliers de tonnes,21502.00,Données standardisées
37491,68,France,Disponibilité intérieure,Blé,2511,Milliers de tonnes,20298.00,Données standardisées
37492,68,France,Aliments pour animaux,Blé,2511,Milliers de tonnes,7822.00,Données standardisées
37493,68,France,Semences,Blé,2511,Milliers de tonnes,748.00,Données standardisées
37494,68,France,Pertes,Blé,2511,Milliers de tonnes,358.00,Données standardisées
37495,68,France,Traitement,Blé,2511,Milliers de tonnes,1575.00,Données standardisées
37496,68,France,Autres utilisations (non alimentaire),Blé,2511,Milliers de tonnes,2824.00,Données standardisées


In [10]:
# je stock mes valeurs dans des variables pour ensuite les calculer
df_vegetaux_élément = df_vegetaux_q2_france['élément']
production = df_vegetaux_q2_france[df_vegetaux_élément == 'Production']['quantite'].sum()
importation = df_vegetaux_q2_france[df_vegetaux_élément == 'Importations - Quantité']['quantite'].sum()
variation = df_vegetaux_q2_france[df_vegetaux_élément == 'Variation de stock']['quantite'].sum()
exportation = df_vegetaux_q2_france[df_vegetaux_élément == 'Exportations - Quantité']['quantite'].sum()
dispo_int = df_vegetaux_q2_france[df_vegetaux_élément == 'Disponibilité intérieure']['quantite'].sum()

# j'effectue mes calculs
print('La disponibilité intérieure vaut :{}'.format(dispo_int))
print('production + importation + variation - exportation vaut : {}'.format((importation + variation + production) - exportation))
 
assert dispo_int == ((production + importation + variation) - exportation), 'La formule est fausse'

La disponibilité intérieure vaut :20298.0
production + importation + variation - exportation vaut : 20298.0


In [11]:
# maintenant il faut preciser à quoi correspond la Disponibilité intérieure de la liste
alim_ani = df_vegetaux_q2_france[df_vegetaux_élément == 'Aliments pour animaux']['quantite'].sum()
semences = df_vegetaux_q2_france[df_vegetaux_élément == 'Semences']['quantite'].sum()
pertes = df_vegetaux_q2_france[df_vegetaux_élément == 'Pertes']['quantite'].sum()
traitement = df_vegetaux_q2_france[df_vegetaux_élément == 'Traitement']['quantite'].sum()
autre_uti = df_vegetaux_q2_france[df_vegetaux_élément == 'Autres utilisations (non alimentaire)']['quantite'].sum()
nourriture = df_vegetaux_q2_france[df_vegetaux_élément == 'Nourriture']['quantite'].sum()

# j'effectue mes calculs
print('La disponibilité intérieure vaut :{}'.format(dispo_int))
print('alim_ani + semences + pertes + traitement + autre_uti + nourriture vaut : {}'.format(alim_ani + semences + pertes + traitement + autre_uti + nourriture))

# cette commande 'assert' me permet de verifier si mon calcul est exact, sinon il renvoie la phrase 'La formule est fausse'.
assert dispo_int == (alim_ani + semences + pertes + traitement + autre_uti + nourriture), 'La formule est fausse'

La disponibilité intérieure vaut :20298.0
alim_ani + semences + pertes + traitement + autre_uti + nourriture vaut : 20298.0


## Donc on peut en conclure que :
disponibilitéIntérieure=(Production+Importation+VariationDeStock)-Exportation

disponibilitéIntérieure=(Alimentspouranimaux+Semences+Pertes+Traitement+Autresutilisations+Nourriture)

disponibilitéIntérieure=(Alimentspouranimaux+Semences+Pertes+Traitement+Autresutilisations+Nourriture)=(Production+Importation+VariationDeStock)-Exportation

## Question 3 : Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.

In [12]:
# je dois filtrer la database de 'df_animal' pour qu'il soit similaire à celui de 'df_vegetaux' pour pouvoir les concatener
df_animal.columns = df_animal.columns = df_animal.columns.str.strip().str.lower().str.replace(' ', '_')
df_animal.rename(columns={'zone': 'pays', 'valeur': 'quantite', 'description_du_symbole': 'symbol'}, inplace=True)
df_animal_q3 = df_animal[['code_zone', 'pays', 'élément', 'produit', 'code_produit', 'unité', 'quantite', 'symbol']]

# supprimer les données redondant 'chine'
df_animal_q3.drop(df_animal_q3[df_animal_q3['code_zone'] == 351].index, inplace=True)

# je concatene mes 2 dataframes
ani_veg_q3 = pd.concat([df_animal_q3, df_vegetaux_q2])

# il va falloir créer un pivot table avec les dataframe (animal et vegetal) tout en rajoutant la colonne 'population' de la dataframe 'df_population'
ani_veg_q3 = ani_veg_q3.pivot_table(
    index = ['code_zone', 'pays', 'code_produit', 'produit'],
    columns = ['élément'], values = ['quantite'], aggfunc = sum)

# je renomme mes colonnes pour pouvoir les identifier pour mes opération plus tard
ani_veg_q3.columns = ['Aliments pour animaux', 'Autres utilisations (non alimentaire)', 'Disponibilité alimentaire (Kcal/personne/jour)',
    'Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité de matière grasse en quantité (g/personne/jour)', 
    'Disponibilité de protéines en quantité (g/personne/jour)', 'Disponibilité intérieure', 'Exportations - Quantité',
    'Importations - Quantité', 'Nourriture', 'Pertes',
    'Production', 'Semences', 'Traitement','Variation de stock']

# je reset_index car sinon je perds mon index de ma pivot_table il se transforme en colonne
ani_veg_q3 = ani_veg_q3.reset_index()

# je rajoute ma colonne population via une jointure
ani_veg_q3 = pd.merge(ani_veg_q3, df_population_q1[['code_zone', 'population']], on = ['code_zone'])

# je convertie ma colonne population pour avoir un nombre réel
ani_veg_q3['population'] = ani_veg_q3['population'] * 1000

ani_veg_q3.head()

C:\Users\lilop\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,code_zone,pays,code_produit,produit,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,population
0,1,Arménie,2511,Blé,93.0,0.0,1024.0,130.60,3.60,30.52,554.0,1.0,361.0,389.0,32.0,312.0,30.0,10.0,-118.0,2977000
1,1,Arménie,2513,Orge,137.0,26.0,0.0,0.00,0.00,0.00,198.0,0.0,9.0,0.0,15.0,189.0,14.0,7.0,0.0,2977000
2,1,Arménie,2514,Maïs,96.0,NaN,0.0,0.03,NaN,0.01,102.0,NaN,82.0,0.0,7.0,21.0,0.0,NaN,NaN,2977000
3,1,Arménie,2515,Seigle,1.0,NaN,1.0,0.12,0.00,0.02,1.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,0.0,2977000
4,1,Arménie,2516,Avoine,4.0,NaN,2.0,0.37,0.03,0.09,6.0,NaN,1.0,1.0,0.0,5.0,0.0,NaN,NaN,2977000


### Voici le calcul à faire pour répondre à la question

In [13]:
# je convertie mes colonnes pour n'avoir qu'une seule données pour l'année et ne pas faire d'autre opération par la suite
ani_veg_q3['Disponibilité alimentaire (Kcal/pays/an)'] = ani_veg_q3['Disponibilité alimentaire (Kcal/personne/jour)'] * ani_veg_q3['population'] * 365
ani_veg_q3['Disponibilité de protéines en quantité (g/produit/pays/an)'] = ani_veg_q3['Disponibilité de protéines en quantité (g/personne/jour)'] / 1000 * ani_veg_q3['population'] * 365
ani_veg_q3.head()

,code_zone,pays,code_produit,produit,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),...,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,population,Disponibilité alimentaire (Kcal/pays/an),Disponibilité de protéines en quantité (g/produit/pays/an)
0,1,Arménie,2511,Blé,93.0,0.0,1024.0,130.60,3.60,30.52,...,361.0,389.0,32.0,312.0,30.0,10.0,-118.0,2977000,1.112684e+12,33163184.60
1,1,Arménie,2513,Orge,137.0,26.0,0.0,0.00,0.00,0.00,...,9.0,0.0,15.0,189.0,14.0,7.0,0.0,2977000,0.000000e+00,0.00
2,1,Arménie,2514,Maïs,96.0,NaN,0.0,0.03,NaN,0.01,...,82.0,0.0,7.0,21.0,0.0,NaN,NaN,2977000,0.000000e+00,10866.05
3,1,Arménie,2515,Seigle,1.0,NaN,1.0,0.12,0.00,0.02,...,0.0,0.0,0.0,1.0,0.0,NaN,0.0,2977000,1.086605e+09,21732.10
4,1,Arménie,2516,Avoine,4.0,NaN,2.0,0.37,0.03,0.09,...,1.0,1.0,0.0,5.0,0.0,NaN,NaN,2977000,2.173210e+09,97794.45


## Question 4 : calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg.

In [14]:
# je dois convertir ma colonne 'nourriture' en kg car il est actuellement en milliers de tonnes
ani_veg_q4 = ani_veg_q3
ani_veg_q4['Disponibilité alimentaire en kg'] = ani_veg_q3['Nourriture'] * 1000000

# je crée un 'mask'qui va me permettre d'ignorer les valeurs nuls de ma dataframe car ils peuvent érroné mes calculs
# ce 'mask' me permet de calculer des valeurs differents de 0 si ce n'est pas le cas mon calcul me retourne un 'NaN'
mask = ani_veg_q4['Disponibilité alimentaire en kg'] != 0

# je crée une colonne qui va contenir mon calcul pour pouvoir répondre à ma question
ani_veg_q4.loc[mask, 'Ratio energie/poids'] = ani_veg_q4.loc[mask, 'Disponibilité alimentaire (Kcal/pays/an)'] / ani_veg_q4.loc[mask, 'Disponibilité alimentaire en kg']

# pour les proteines je vais proceder de la même manières en évitant les calculs des nombres avec 0 comme facteur
# en multipliant par 100 pour avoir un chiffre plus facile à lire
ani_veg_q4.loc[mask, '% de proteines'] = ani_veg_q4.loc[mask, 'Disponibilité de protéines en quantité (g/produit/pays/an)'] / ani_veg_q4.loc[mask, 'Disponibilité alimentaire en kg'] * 100

ani_veg_q4.head(5)

,code_zone,pays,code_produit,produit,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),...,Production,Semences,Traitement,Variation de stock,population,Disponibilité alimentaire (Kcal/pays/an),Disponibilité de protéines en quantité (g/produit/pays/an),Disponibilité alimentaire en kg,Ratio energie/poids,% de proteines
0,1,Arménie,2511,Blé,93.0,0.0,1024.0,130.60,3.60,30.52,...,312.0,30.0,10.0,-118.0,2977000,1.112684e+12,33163184.60,389000000.0,2860.368946,8.525240
1,1,Arménie,2513,Orge,137.0,26.0,0.0,0.00,0.00,0.00,...,189.0,14.0,7.0,0.0,2977000,0.000000e+00,0.00,0.0,NaN,NaN
2,1,Arménie,2514,Maïs,96.0,NaN,0.0,0.03,NaN,0.01,...,21.0,0.0,NaN,NaN,2977000,0.000000e+00,10866.05,0.0,NaN,NaN
3,1,Arménie,2515,Seigle,1.0,NaN,1.0,0.12,0.00,0.02,...,1.0,0.0,NaN,0.0,2977000,1.086605e+09,21732.10,0.0,NaN,NaN
4,1,Arménie,2516,Avoine,4.0,NaN,2.0,0.37,0.03,0.09,...,5.0,0.0,NaN,NaN,2977000,2.173210e+09,97794.45,1000000.0,2173.210000,9.779445


### vérification du ratio energie/poids avec l'oeuf comme exemple

In [15]:
# je verifie mes calculs en prenent comme exemple l'Oeufs, ou je vais prendre toutes les valeurs et en faire une moyenne et
# comparer cette valeurs avec wikipedia, wiki => 147 Kcal pour 100g
ani_veg_q4[ani_veg_q4.produit == 'Oeufs']['Ratio energie/poids'].mean()

1345.3003859252408

### vérification du pourcentage de proteines avec l'avoine comme exemple

In [16]:
# je verifie mes calculs en prenent comme exemple l'avoine, ou je vais prendre toutes les valeurs et en faire une moyenne et
# comparer cette valeurs avec wikipedia, wiki => 10.7g pour 100g
ani_veg_q4[ani_veg_q4.produit == 'Avoine']['% de proteines'].mean()

8.17749336154237

## Question 5 : Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids.

In [17]:
# crée une nouvelle dataframe avec comme index les produit et la colonne Ratio energie/poids pour les plus calorifique
# et la colonne % de proteines pour les plus riches en proteines
df_q5_ratio = ani_veg_q4.groupby('produit')['Ratio energie/poids'].mean()
df_q5_prot = ani_veg_q4.groupby('produit')['% de proteines'].mean()

# maintenent que j'ai mes valeurs il faut les ajouter dans une dataframe
df_ratio_q5 = pd.DataFrame({'Ratio energie/poids': df_q5_ratio, '% de proteines': df_q5_prot})

# je vais les classé par ordre descroissant ma colonne 'Ratio energie/poids'
ratio_q5 = df_ratio_q5.sort_values('Ratio energie/poids', ascending=False)
ratio_q5.head(5)

,Ratio energie/poids,% de proteines
produit,,
Huiles de Poissons,9003.715504,0.000000
Huile de Palmistes,8789.540277,0.304681
Huile de Palme,8719.419872,0.386137
Huile de Germe de Maïs,8649.968502,0.187364
Huile de Son de Riz,8641.430661,0.692650


In [18]:
# je vais classé par ordre descroissant ma colonne '% de proteines'
prot_q5 = df_ratio_q5.sort_values('% de proteines', ascending=False)
prot_q5.head(5)

,Ratio energie/poids,% de proteines
produit,,
Soja,2376.843544,28.533053
Arachides Decortiquees,5620.249374,25.146613
Pois,3443.737407,22.324375
Légumineuses Autres,3339.891537,21.806434
Haricots,3341.783779,21.545181


## Question 6 : Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [19]:
# je rajoute ma colonne 'population' dans ma nouvelle dataframe
df_veg_q6 = df_vegetaux.merge(df_population_q1, on='pays')

# maintenent je vais selectionner dans ma dataframe les ligne qui comporte 'Disponibilité alimentaire' dans la colonne 'élément'
df_veg_q6 = df_veg_q6[df_veg_q6['élément'] == 'Disponibilité intérieure'].copy()

# je convertie ma colonne 'quantite' pour avoir un chiffre réel et pouvoir le calculer sous la bonne unité de mesure
df_veg_q6['quantite(kg)'] = df_veg_q6['quantite'] * 1000000

# je vais prendre mes colonnes qui vont m'aider pour mes calculs
df_veg_q6 = df_veg_q6[['produit', 'quantite(kg)']].groupby('produit').sum()

# je rajoute mes facteurs de calculs qui vont me permettre de répondre à la question que j'ai calculé à la question précedente
df_veg_q6 = df_veg_q6.merge(df_ratio_q5, on='produit')

# ensuite je vais pouvoir effectuer mes calculs, je vais multiplier ma colonne 'quantite(kg)' par 'Ratio energie/poids'
df_veg_q6['total_kcal'] = df_veg_q6['quantite(kg)'] * df_veg_q6['Ratio energie/poids']

# je vais additionner toutes les valeurs de tout les produits pour avoir la disponibilité interieur mondial
total_vegetaux_kcal = df_veg_q6['total_kcal'].sum()
df_veg_q6.head()

,quantite(kg),Ratio energie/poids,% de proteines,total_kcal
produit,,,,
"Agrumes, Autres",1.217600e+10,90.316654,0.216705,1.099696e+12
"Alcool, non Comestible",2.174100e+10,NaN,NaN,NaN
Aliments pour enfants,8.400000e+07,3224.621937,14.726562,2.708682e+11
Ananas,2.328200e+10,366.683059,0.236350,8.537115e+12
Arachides Decortiquees,2.969500e+10,5620.249374,25.146613,1.668933e+14


### Voici le résultat de la disponibilité interieur uniquement pour les produits vegetaux

In [20]:
# je divise le nombre par 1 000 000 car en kg le chiffre est difficile à comprendre
print("Le total de kcal disponible uniquement grâce aux végetaux s'éléve à : {}".format(total_vegetaux_kcal))

Le total de kcal disponible uniquement grâce aux végetaux s'éléve à : 1.3095798424730494e+16


## Question 7 : Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

### Kcal (végétaux)

In [21]:
# j'identifie les differents facteurs qui pourront m'aider pour mes calculs
jours = 365
moyenne_kcal_jours = 2500

# j'effectue mes calculs en  prenent mon total de kcal que je vais diviser par la multiplication de la moyenne de kcal depenser par
# jours et le nombre de jours (nb_habitants_nourri = total_kcal_de_vegetaux / (365*2500)
nb_habitants_nourri_kcal = total_vegetaux_kcal / (moyenne_kcal_jours * jours)

# Pour avoir un pourcentage il faut diviser le facteurs de la population mondiale (pourcentage = total_kcal_de_vegetaux / (365*2500) / world_pop)
pourcentage_habitants_nourri = total_vegetaux_kcal / (moyenne_kcal_jours * jours) / world_pop

# j'affiche mon résultat
print('On pourrait nourrir {0:,.0f} de la population mondiale uniquement avec la disponibilité intérieure en végétaux'.format(nb_habitants_nourri_kcal))
print('Le pourcentage de personne pouvant être nourri dans le monde grâce à la disponibilité intérieure en végétaux est de : {0:.0f}%'.format(pourcentage_habitants_nourri * 100))

On pourrait nourrir 14,351,559,918 de la population mondiale uniquement avec la disponibilité intérieure en végétaux
Le pourcentage de personne pouvant être nourri dans le monde grâce à la disponibilité intérieure en végétaux est de : 205%


### proteines (végétaux)

In [22]:
# je vais calculer la totalité de proteines disponible grâce aux vegetaux dans le monde
df_veg_q6['total_prot_produit'] = df_veg_q6['quantite(kg)'] * df_veg_q6['% de proteines'] / 100

# je vais additionner toutes mes valeurs pour avoir le total de prot disponible avec les vegetaux
total_vegetaux_prot = df_veg_q6['total_prot_produit'].sum()

In [23]:
# je vais proceder de la même manière qu'avec les kcal seulement je vais convertir le poids en kg
proteines = 56

# j'effectue mes calculs en  prenent mon total de kcal que je vais diviser par la multiplication de la moyenne de proteines
# depenser par jours que je vais convertir en kg et le nombre de jours (nb_habitants_nourri = total_kcal_de_vegetaux / (365*60/0.001)
nb_habitants_nourri_prot = total_vegetaux_prot / (proteines * 0.001 * jours)

# Je calcul mon pourcentage de proteines disponible par habitants dans le monde en 2013
pourcentage_habitants_nourri_prot = total_vegetaux_prot / (proteines * 0.001 * jours) / world_pop

# j'affiche mon résultat
print("Le nombre de protéines disponible grace aux végétaux s'éléve à : {0:,.0f}".format(nb_habitants_nourri_prot))
print("Le pourcentage de protéines disponible par habitants grâce aux végétaux s'éléve à : {0:.0f}%".format(pourcentage_habitants_nourri_prot * 100))

Le nombre de protéines disponible grace aux végétaux s'éléve à : 15,934,358,854
Le pourcentage de protéines disponible par habitants grâce aux végétaux s'éléve à : 228%


## Question 8 : Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux, la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [24]:
# je vais joindre ma colonne population dans ma dataframe vegetaux
veg_q8 = df_vegetaux.merge(df_population_q1, on='pays').copy()

# Je convertie ma colonne 'population' et 'quantite(kg)'
veg_q8['population'] = veg_q8['population'] * 1000
veg_q8['quantite(kg)'] = veg_q8['quantite'] * 1000000

# je filtre ma dataframe avec les éléments utiles pour mes calculs
veg_q8 = veg_q8[veg_q8['élément'].isin(['Aliments pour animaux', 'Nourriture', 'Pertes'])]

# Je selectionne les colonnes 'quantité' et 'produit' et je vais indexer cette derniere pour avoir une ligne = 1 produit
veg_q8_calcul = veg_q8[['quantite(kg)', 'produit']].groupby('produit').sum()

# je rajoute a cette nouvelle dataframe mes facteurs de la question 5 pour pouvoir effectuer mes calculs
veg_q8_calcul = veg_q8_calcul.merge(ratio_q5, on='produit')

# j'effectue mes calculs comme pour la question 6
veg_q8_calcul['total_kcal'] = veg_q8_calcul['quantite(kg)'] * veg_q8_calcul['Ratio energie/poids']
veg_q8_calcul['total_prot'] = veg_q8_calcul['quantite(kg)'] * veg_q8_calcul['% de proteines'] / 100

# je vais additionner toutes ces dernieres valeurs pour avoir la disponibilité alimentaire total mondiale en végétaux (nourriture, aliments pour animaux, pertes)
veg_q8_total_kcal = veg_q8_calcul['total_kcal'].sum()
veg_q8_total_prot = veg_q8_calcul['total_prot'].sum()

# j'affiche mes résultats
print("La disponibilité alimentaire mondiale (Nourriture/Pertes/Aliments pour animaux) en kcal s'éléve à {0:,}".format(veg_q8_total_kcal))
print("La disponibilité alimentaire mondiale (Nourriture/Pertes/Aliments pour animaux) en protéines s'éléve à {0:,}".format(veg_q8_total_prot))

La disponibilité alimentaire mondiale (Nourriture/Pertes/Aliments pour animaux) en kcal s'éléve à 9,287,743,876,331,156.0
La disponibilité alimentaire mondiale (Nourriture/Pertes/Aliments pour animaux) en protéines s'éléve à 209,732,894,974.1186


### Le pourcentage de la disponibilité alimentaire via la Nourriture, les Pertes et les Aliments pour animaux

In [25]:
# je vais proceder de la même manieres qu'avec la question 7
# je peux deja proceder à mes calculs en kcal
habitants_nourri_kcal_pertes = veg_q8_total_kcal / (moyenne_kcal_jours * jours)
pourcentage_habitants_nourri_kcal_pertes = veg_q8_total_kcal / (moyenne_kcal_jours * jours) / world_pop

# en proteines
habitants_nourri_prot_pertes = veg_q8_total_prot / (proteines * 0.001 * jours)
pourcentage_nourri_prot_pertes = (habitants_nourri_prot_pertes / world_pop) * 100

# j'affiche mes résultats
print("On pourrait nourrir {0:,.0f} de la population mondiale si on ne donnait pas d'aliments aux animaux ni de pertes".format(habitants_nourri_kcal_pertes))
print("Le pourcentage de personne pouvant être nourri dans le monde grâce à la disponibilité intérieure en végétaux est de : {0:.2f}%".format(pourcentage_habitants_nourri_kcal_pertes * 100))
print('------------------------------------------------------------------------')
print("Le nombre de protéines disponibles sans aliments pour animaux ni de pertes s'éléve à : {0:,.0f}".format(habitants_nourri_prot_pertes))
print("Le pourcentage de protéines disponible par habitants grâce aux végétaux s'éléve à : {0:.0f}%".format(pourcentage_nourri_prot_pertes))

On pourrait nourrir 10,178,349,454 de la population mondiale si on ne donnait pas d'aliments aux animaux ni de pertes
Le pourcentage de personne pouvant être nourri dans le monde grâce à la disponibilité intérieure en végétaux est de : 145.46%
------------------------------------------------------------------------
Le nombre de protéines disponibles sans aliments pour animaux ni de pertes s'éléve à : 10,260,904,842
Le pourcentage de protéines disponible par habitants grâce aux végétaux s'éléve à : 147%


## Question 9 : Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [26]:
# je vais concatener 2 dataframes et rajouter ma colonne 'population' à cette dernière
ani_veg_q9 = df_population.merge(pd.concat([df_vegetaux, df_animal]), on='pays')

# je selectionne uniquement les lignes comportant la valeur 'Nourriture' de ma dataframe
ani_veg_q9 = ani_veg_q9[ani_veg_q9['élément_y'].isin(['Nourriture'])]

# je vais convertir mes valeurs de la colonne 'quantite' et 'population'
ani_veg_q9['quantite(kg)'] = ani_veg_q9['quantite'] * 1000000
ani_veg_q9['population'] = ani_veg_q9['population'] * 1000

# je rename ma colonne 'produit_y' car sinon je ne pourrais pas joindre mes facteurs de calcul (question 5)
ani_veg_q9.rename(columns={'produit_y': 'produit', 'code_zone_y': 'code_zone'}, inplace=True)

# Je filtre mes données pour supprimer les valeurs contenent 'Chine' dans la colonne 'pays' qui est compter 2 fois
ani_veg_q9.drop(ani_veg_q9[ani_veg_q9['code_zone'] == 351].index, inplace=True)

# Je selectionne les colonnes 'quantité' et 'produit' et je vais indexer cette derniere pour avoir une ligne = 1 produit
ani_veg_q9 = ani_veg_q9[['quantite(kg)', 'produit']].groupby('produit').sum()

# je rajoute a cette nouvelle dataframe mes facteurs de la question 5 pour pouvoir effectuer mes calculs
ani_veg_q9_calcul = ani_veg_q9.merge(ratio_q5, on='produit')

# j'effectue mes calculs comme pour la question 6
ani_veg_q9_calcul['total_kcal'] = ani_veg_q9_calcul['quantite(kg)'] * ani_veg_q9_calcul['Ratio energie/poids']
ani_veg_q9_calcul['total_prot'] = ani_veg_q9_calcul['quantite(kg)'] * ani_veg_q9_calcul['% de proteines']

# j'additione mes valeurs ensemble
ani_veg_total_kcal = ani_veg_q9_calcul['total_kcal'].sum()
ani_veg_total_prot = ani_veg_q9_calcul['total_prot'].sum() / 100

# j'affiche mes résultats
print("La disponibilité alimentaire mondiale en kcal s'élève à {0:,.2f} Kcal".format(ani_veg_total_kcal))
print("La disponibilité alimentaire mondiale en protéines s'élève à {0:,.2f} Kg".format(ani_veg_total_prot))

La disponibilité alimentaire mondiale en kcal s'élève à 7,201,684,086,414,998.00 Kcal
La disponibilité alimentaire mondiale en protéines s'élève à 202,345,720,685.93 Kg


In [27]:
ani_veg_q9.head()

,quantite(kg)
produit,
Abats Comestible,1.566900e+10
"Agrumes, Autres",1.135900e+10
Aliments pour enfants,7.610000e+08
Ananas,2.075700e+10
Animaux Aquatiques Autre,1.249000e+09


### convertir ces résultats en pourcentage

In [28]:
# je vais utiliser la même méthode qu'avec la question 8
# je peux deja proceder à mes calculs en kcal
habitants_nourri_kcal_mondiale = ani_veg_total_kcal / (moyenne_kcal_jours * jours)
pourcentage_habitants_nourri_kcal_mondiale = (habitants_nourri_kcal_mondiale / world_pop)

# en proteines
habitants_nourri_prot_mondiale = ani_veg_total_prot / (proteines * 0.001* jours)
pourcentage_nourri_prot_mondiale = (habitants_nourri_prot_mondiale / world_pop) * 100

# j'affiche mes résultats
print("On pourrait nourrir {0:,.0f} de la population avec la disponibilité alimentaire mondiale".format(habitants_nourri_kcal_mondiale))
print("Le pourcentage de personne pouvant être nourri dans le monde grâce à la disponibilité alimentaire mondiale est de : {0:.0f}%".format(pourcentage_habitants_nourri_kcal_mondiale * 100))
print('------------------------------------------------------------------------')
print("Le nombre de protéines avec la disponibilité alimentaire mondiale s'élève à : {0:,.2f}".format(habitants_nourri_prot_mondiale))
print("Le pourcentage de protéines disponible avec la disponibilité alimentaire mondiale s'élève à : {0:.0f}%".format(pourcentage_nourri_prot_mondiale))

On pourrait nourrir 7,892,256,533 de la population avec la disponibilité alimentaire mondiale
Le pourcentage de personne pouvant être nourri dans le monde grâce à la disponibilité alimentaire mondiale est de : 113%
------------------------------------------------------------------------
Le nombre de protéines avec la disponibilité alimentaire mondiale s'élève à : 9,899,497,098.14
Le pourcentage de protéines disponible avec la disponibilité alimentaire mondiale s'élève à : 141%


## Question 10 : A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [29]:
# je filtre ma dataframe
df_sousalimentation.columns = df_sousalimentation.columns = df_sousalimentation.columns.str.strip().str.lower().str.replace(' ', '_')
df_sousalimentation.rename(columns={'zone': 'pays', 'valeur': 'population', 'description_du_symbole': 'symbol'}, inplace=True)

# je selectionne uniquement les lignes pour l'année 2013 (2012-2014)
df_sousalimentation_q10 = df_sousalimentation[df_sousalimentation['année'] == '2012-2014'].copy()

# je filtre à nouveaux ma nouvelle dataframe pour supprimer les valeurs nuls pour ne pas fausser mon résultat
df_sousalimentation_q10['population'] = pd.to_numeric(df_sousalimentation_q10['population'], errors='coerce')

# je convertie ma colonne 'population' pour avoir un nombre réel
df_sousalimentation_q10['population'] = df_sousalimentation_q10['population'] * 1000000

# je vais additionner toutes la colonne ensemble puis la diviser par la population mondiale
df_sousalimentation_q10_calcul = df_sousalimentation_q10['population'].sum()
total_sous_nutrition = df_sousalimentation_q10_calcul / world_pop * 100

# j'affiche mes mon résultat
print('Le pourcentage de personnes en sous-nutrition dans le monde est de {0:.1f}%'.format(total_sous_nutrition))

Le pourcentage de personnes en sous-nutrition dans le monde est de 12.6%


### Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO

In [30]:
# je filtre ma dataframe 'df_cereales' pour qu'elle soit identique avec 'ani_veg_q3'
df_cereales.columns = df_cereales.columns = df_cereales.columns.str.strip().str.lower().str.replace(' ', '_')
df_cereales.rename(columns={'zone': 'pays', 'valeur': 'population', 'description_du_symbole': 'symbol'}, inplace=True)

In [31]:
# je demande à ma dataframe 'veg_q10' si il trouve la même valeur que dans la dataframe 'df_cereales' dans la colonne produit
# et si c'est le cas de me retourner un valeur 'True'
veg_q10 = ani_veg_q3.copy()
veg_q10['is_cereale'] = veg_q10['produit'].isin(list(df_cereales['produit']))

## Question 11 : En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [32]:
# je selectionne les données 'exportation' de ma derniere dataframe et je vais les ranger par ordre descroissant
veg_q11_cereale = veg_q10[veg_q10['is_cereale']]

# je vais filtrer ma dataframe car il y a des valeurs nul qui peuvent faussé mon résultat
veg_q11_nourriture = veg_q11_cereale[(veg_q11_cereale['Aliments pour animaux'] > 0) | (veg_q11_cereale['Nourriture'] > 0)]

# ensuite je vais additionner ces deux colonnes ensemble
veg_q11_total_nourriture = veg_q11_nourriture['Nourriture'].sum()
veg_q11_total_animaux = veg_q11_nourriture['Aliments pour animaux'].sum()
veg_q11_total = veg_q11_total_nourriture + veg_q11_total_animaux

# je vais diviser mon dernier résultat par la disponibilité alimentaire animal pour savoir la proportion destinée à l'alimentation animale
veg_q11_proportion = (veg_q11_total_animaux / veg_q11_total) * 100

# j'affiche mon résultat
print("La proportion destinée à l'alimentation animale en terme de poids est de {0:.0f}%".format(veg_q11_proportion))

La proportion destinée à l'alimentation animale en terme de poids est de 46%


### Repérez les 20 produits les plus exportés par ce groupe de pays.

In [33]:
# je vais filtrer ma dataframe de la question 10 pour supprimer les valeurs inexistante
df_exportation_q11 = df_sousalimentation_q10[df_sousalimentation_q10['population'] > 0]

# je convertie ma colonne population
df_exportation_q11 = df_exportation_q11.merge(ani_veg_q3, left_on='code_zone', right_on='code_zone')

# je vais créer un nouveau dataframe, que je vais aggreger par produit et prendre la colonne
# 'exportation' que je vais additionné par produit
df_exportation_q11 = df_exportation_q11[['produit_y', 'Exportations - Quantité']].groupby('produit_y').sum()

# je vais classé par ordre descroissant
df_exportation_q11 = df_exportation_q11.sort_values(by='Exportations - Quantité', ascending=False)[:20]

# j'affiche mon résultat
df_exportation_q11

,Exportations - Quantité
produit_y,
Huile de Palme,46285.0
Maïs,37888.0
Manioc,35843.0
Riz (Eq Blanchi),32787.0
Sucre Eq Brut,25237.0
Blé,24841.0
"Légumes, Autres",19948.0
Bananes,17805.0
"Fruits, Autres",14590.0


### Sélectionnez les 200 plus grandes importations de ces produits

In [34]:
# je réintialise mon index
df_exportation_q11.reset_index(inplace=True)

# je verifie si les données de mes deux colonnes 'produit' corresponde
importation_q11 = ani_veg_q3[ani_veg_q3['produit'].isin(df_exportation_q11['produit_y'])]

# je classe dans l'ordre descroissant en prenant les 200 premieres importation
importation_q11 = importation_q11.sort_values(by='Importations - Quantité', ascending=False)[:200]

# j'affiche mon résultat
pd.set_option('max_columns', None)
importation_q11.head(20)

,code_zone,pays,code_produit,produit,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,population,Disponibilité alimentaire (Kcal/pays/an),Disponibilité de protéines en quantité (g/produit/pays/an),Disponibilité alimentaire en kg,Ratio energie/poids,% de proteines
2509,41,"Chine, continentale",2555,Soja,9530.0,NaN,35.0,3.66,1.18,3.29,75081.0,250.0,63381.0,5072.0,604.0,11951.0,713.0,59162.0,0.0,1385567000,1.770062e+13,1.663858e+09,5.072000e+09,3489.869563,32.804774
2496,41,"Chine, continentale",2532,Manioc,22868.0,7940.0,6.0,1.91,0.01,0.05,33590.0,41.0,29046.0,2645.0,138.0,4585.0,NaN,NaN,NaN,1385567000,3.034392e+12,2.528660e+07,2.645000e+09,1147.218045,0.956015
6965,110,Japon,2514,Maïs,10964.0,288.0,61.0,9.21,0.20,0.16,14661.0,1.0,14403.0,1171.0,3.0,0.0,0.0,2235.0,259.0,127144000,2.830861e+12,7.425210e+06,1.171000e+09,2417.473237,0.634091
12012,194,Arabie saoudite,2513,Orge,10000.0,NaN,0.0,0.01,0.00,0.00,10317.0,0.0,10547.0,0.0,317.0,11.0,0.0,NaN,-241.0,28829000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
3674,59,Égypte,2511,Blé,4860.0,0.0,1175.0,146.83,6.27,35.62,19341.0,191.0,10331.0,12048.0,2224.0,9460.0,217.0,0.0,-260.0,82056000,3.519177e+13,1.066835e+09,1.204800e+10,2920.963396,8.854869
7490,117,République de Corée,2514,Maïs,6281.0,47.0,82.0,12.46,0.17,0.45,8749.0,86.0,8755.0,614.0,176.0,80.0,1.0,1631.0,0.0,49263000,1.474442e+12,8.091448e+06,6.140000e+08,2401.370668,1.317825
6069,100,Inde,2577,Huile de Palme,NaN,6782.0,31.0,1.30,3.54,NaN,8413.0,143.0,8568.0,1631.0,NaN,NaN,NaN,NaN,-12.0,1252140000,1.416796e+13,NaN,1.631000e+09,8686.673268,NaN
4769,79,Allemagne,2848,Lait - Excl Beurre,2064.0,504.0,331.0,258.70,16.50,24.58,23999.0,15677.0,8332.0,21401.0,30.0,31345.0,NaN,0.0,0.0,82727000,9.994663e+12,7.422018e+08,2.140100e+10,467.018481,3.468071
2581,41,"Chine, continentale",2848,Lait - Excl Beurre,1608.0,NaN,58.0,32.66,3.22,3.07,48271.0,129.0,8207.0,45252.0,1410.0,40193.0,NaN,0.0,0.0,1385567000,2.933245e+13,1.552597e+09,4.525200e+10,648.202364,3.431002
6681,106,Italie,2848,Lait - Excl Beurre,524.0,301.0,272.0,246.88,18.27,17.07,15918.0,2781.0,7695.0,15057.0,36.0,11004.0,NaN,0.0,0.0,60990000,6.055087e+12,3.800012e+08,1.505700e+10,402.144332,2.523751


### Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :

#### le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
#### le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

In [35]:
# je vais aggreger ma dataframe par produit
produit_q11 = importation_q11.groupby('produit').sum()

# je vais calculer mon 1er ratio que je vais stocker dans une nouvelle colonne "Autres utilisations" / "disponibilité intérieure"
produit_q11['Autre/Dispo_int'] = produit_q11['Autres utilisations (non alimentaire)'] / produit_q11['Disponibilité intérieure'] * 100

# je vais calculer mon 2eme ratio que je vais stocker dans une nouvelle colonne ("nourriture animale" + "nourriture") / "nourriture animale"
produit_q11['total_nourriture'] = produit_q11['Aliments pour animaux'] + produit_q11['Nourriture']
produit_q11['Ratio Nourriture/Animal'] = (produit_q11['Aliments pour animaux'] / produit_q11['total_nourriture']) * 100

# j'affiche mon résultat
produit_q11.head(20)

,code_zone,code_produit,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock,population,Disponibilité alimentaire (Kcal/pays/an),Disponibilité de protéines en quantité (g/produit/pays/an),Disponibilité alimentaire en kg,Ratio energie/poids,% de proteines,Autre/Dispo_int,total_nourriture,Ratio Nourriture/Animal
produit,,,,,,,,,,,,,,,,,,,,,,,,,,
Ananas,231,2618,0.0,0.0,7.0,6.35,0.02,0.05,2089.0,131.0,2049.0,2032.0,57.0,171.0,0.0,0.0,0.0,320051000,8.177303e+11,5.840931e+06,2.032000e+09,402.426331,0.287447,0.000000,2032.0,0.000000
Bananes,979,13075,0.0,0.0,89.0,53.45,0.29,1.10,7466.0,2204.0,9663.0,7123.0,344.0,7.0,0.0,0.0,0.0,620100000,4.281632e+12,5.223482e+07,7.123000e+09,3033.521603,3.685994,0.000000,7123.0,0.000000
Blé,5922,102951,95503.0,14363.0,29567.0,3804.70,130.20,896.63,446330.0,103090.0,145447.0,295075.0,15531.0,410379.0,19965.0,5940.0,-6411.0,4247057000,8.599473e+14,2.648487e+10,2.950750e+11,115584.246886,345.776584,3.218023,390578.0,24.451710
Café,310,5260,0.0,0.0,13.0,11.02,0.00,1.76,1926.0,1124.0,2986.0,1926.0,0.0,3.0,0.0,0.0,61.0,402778000,8.256559e+11,1.172450e+08,1.926000e+09,859.414844,11.833465,0.000000,1926.0,0.000000
"Fruits, Autres",866,18375,0.0,0.0,236.0,195.30,3.47,3.12,72851.0,5818.0,15714.0,66962.0,4293.0,62879.0,0.0,1640.0,78.0,2094036000,2.861680e+13,3.524160e+08,6.696200e+10,3103.900728,4.151849,0.000000,66962.0,0.000000
Huile de Palme,1222,25770,0.0,18568.0,755.0,32.06,85.37,0.23,25618.0,22878.0,29747.0,6833.0,224.0,19446.0,0.0,3.0,-698.0,3533616000,5.874067e+13,8.331778e+06,6.833000e+09,69568.085588,1.810603,72.480287,6833.0,0.000000
Huile de Soja,41,2571,0.0,9200.0,43.0,1.77,4.86,0.00,11658.0,90.0,1158.0,2458.0,0.0,10590.0,0.0,0.0,0.0,1385567000,2.174647e+13,0.000000e+00,2.458000e+09,8847.222972,0.000000,78.915766,2458.0,0.000000
Lait - Excl Beurre,3703,71200,26906.0,12274.0,5676.0,4069.83,330.03,356.47,338523.0,72704.0,86254.0,296651.0,3420.0,323295.0,0.0,155.0,1684.0,3142485000,1.602721e+14,9.657138e+09,2.966510e+11,13345.062653,81.166448,3.625751,323557.0,8.315691
"Légumes, Autres",1340,23445,3428.0,0.0,528.0,670.40,4.07,23.31,69943.0,12278.0,26011.0,61338.0,5053.0,56192.0,16.0,160.0,17.0,863476000,1.587096e+13,7.488892e+08,6.133800e+10,2572.216158,11.430097,0.000000,64766.0,5.292901


## Question 12 : Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer).

In [36]:
# je vais reset_index pour pouvoir selectionner ma colonne 'produit'
produit_q11.reset_index(inplace=True)

# je vais selectionner ma colonne 'Autre/Dispo_int' et ma colonne 'produit' que je vais mettre en index
ratio_q12_autre = produit_q11[['produit', 'Autre/Dispo_int']].groupby('produit').sum()

# je vais classer dans l'ordre descroissant
ratio_q12_autre = ratio_q12_autre.sort_values(by='Autre/Dispo_int', ascending=False)[:3]

# j'affiche mon résultat
ratio_q12_autre

,Autre/Dispo_int
produit,
Huile de Soja,78.915766
Huile de Palme,72.480287
Manioc,26.587329


In [37]:
# je fais pareille avec la colonne 'Ratio Nourriture/Animal'
ratio_q12_animal = produit_q11[['produit', 'Ratio Nourriture/Animal']].groupby('produit').sum()

# je vais classer dans l'ordre descroissant
ratio_q12_animal = ratio_q12_animal.sort_values(by='Ratio Nourriture/Animal', ascending=False)[:3]

# j'affiche mon résultat
ratio_q12_animal

,Ratio Nourriture/Animal
produit,
Orge,98.350495
Maïs,87.651007
Manioc,87.506651


## Question 13 : Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [38]:
# je selectionne les lignes de ma dataframe contenent la valeur 'États-Unis d'Amérique' dans ma colonne 'pays'
cereale_amerique_q13 = veg_q11_cereale[veg_q11_cereale.pays == "États-Unis d'Amérique"].copy()

# j'additionne toutes mes valeurs de la colonne 'Aliments pour animaux'
cereale_calcul_q13 = cereale_amerique_q13['Aliments pour animaux'].sum() * 1000

# j'effectue mon calcul pour diminuer la production de produits animaux de 10% 
cereales_q13 = cereale_calcul_q13 / 10

# j'affiche mon résultat
print("On pourrait libérées {0:,.0f} de tonnes si les États-Unis d'Amérique diminuaient leur production de produits animaux de 10%".format(cereales_q13))

On pourrait libérées 14,009,600 de tonnes si les États-Unis d'Amérique diminuaient leur production de produits animaux de 10%


## Question 14 : En Thaïlande, quelle proportion de manioc produit est exportée ? Quelle est la proportion de personnes en sous-nutrition ?

In [39]:
# je selectionne les valeurs qui comprenne 'Thaïlande' et 'Manioc' dans la colonne 'pays' de ma dataframe 'df_vegetaux'
proportion_q14 = ani_veg_q3[(ani_veg_q3.pays == 'Thaïlande') & (ani_veg_q3.code_produit == 2532)].copy()

# je divise ma colonnea 'Exportations - Quantité' par l'addition de (Importation + Production + Variation de stock)
proportion_q14['Proportion_Manioc'] = (proportion_q14['Exportations - Quantité'] / (proportion_q14['Importations - Quantité'] + proportion_q14['Production'] + proportion_q14['Variation de stock'])) * 100

# j'affiche mon résultat (j'ajoute la fonction .sum() car sinon mon résultat n'est pas dans un bon format pour l'afficher)
print("La proportion d'exportation du Manioc en Thaïlande s'éléve à : {0:.1f}%".format(proportion_q14['Proportion_Manioc'].sum()))

La proportion d'exportation du Manioc en Thaïlande s'éléve à : 80.1%


### Quelle est la proportion de personnes en sous-nutrition ?

In [40]:
# je vais prendre la valeur de la population totale en Thaïlande
df_sousalimentation_thailande = df_sousalimentation[(df_sousalimentation.année == '2012-2014') & (df_sousalimentation.pays == 'Thaïlande')]

# je rajoute ma colonne 'population' de ma dataframe 'df_population' pour avoir mon facteur de calcul (population du pays)
df_population_thailande = df_sousalimentation_thailande.merge(df_population_q1, on = 'code_zone')

# je change le format de mes valeurs pour pouvoir les convertir pour mes calculs plus tard
df_population_thailande['population_x'] = pd.to_numeric(df_population_thailande['population_x'], errors='coerce')

# je convertie mes colonnes 'population(df_sousalimentation_thailande)' & 'population(df_population_q1)'pour mon prochain calcul
df_population_thailande['population_x'] = df_population_thailande['population_x'].sum() * 1000000
df_population_thailande['population_y'] = df_population_thailande['population_y'] * 1000

# je vais diviser ces deux nombre pour avoir ma proportion des habitants en sous-nutrition en Thaïlande
df_proportion_q14 = (df_population_thailande['population_x'] / df_population_thailande['population_y']) * 100

# j'affiche mon résultat
print("La proportion de personne en sous-alimentation s'éléve à {0:.2f}% en Thaïlande".format(df_proportion_q14.sum()))

La proportion de personne en sous-alimentation s'éléve à 8.36% en Thaïlande


## Importez nos données dans un SGBR

### Importez la table 'population'

In [41]:
# je selectionne les colonnes demandés de ma df_population que je stock dans une variable
population = df_population[['pays', 'code_zone', 'année', 'population']]

# j'effectue la manipulation que me permet d'enregistrer ma variable en format .csv
population.to_csv("export.csv", index = False)

### Importez la table 'dispo_alim'

In [42]:
# j'organise ma dataframe pour pouvoir la telecharger

# je joint les df_animal et df_vegetaux pour avoir ma disponibilité alimentaire
dispo_alim = ani_veg_q3.copy()

# je reprecise la nature de mes vegetaux pour savoir si c'est du cereale ou non
dispo_alim['is_cereale'] = dispo_alim['produit'].isin(list(df_cereales['produit']))

# je precise la nature des produits
dispo_alim.loc[dispo_alim['produit'].isin(df_animal['produit']), 'nature_du_produit'] = 'animal'
dispo_alim.loc[dispo_alim['produit'].isin(df_vegetaux['produit']), 'nature_du_produit'] = 'vegetal'

# je rajoute la colonne année dans ma dataframe
dispo_alim['année'] = 2013

# j'ajoute ma colonne 'diponibilité alimentaire'
dispo_alim['disponibilité alimentaire'] = dispo_alim['Nourriture'] * 1000


# je modifie mes colonnes pour uniquement avoir ceux demandés
dispo_alim = dispo_alim[['pays', 'code_zone', 'année', 'produit', 'code_produit',  'nature_du_produit', 
                         'disponibilité alimentaire',
                         'Disponibilité alimentaire (Kcal/personne/jour)', 
                         'Disponibilité de protéines en quantité (g/personne/jour)',
                         'Disponibilité de matière grasse en quantité (g/personne/jour)', ]]

# je rename mes colonnes
dispo_alim.rename(columns={'code_zone': 'code_pays', 'nature_du_produit': 'origin', 'disponibilité alimentaire': 'dispo_alim_tonnes', 'Disponibilité alimentaire (Kcal/personne/jour)': 'dispo_alim_kcal_p_j', 'Disponibilité de matière grasse en quantité (g/personne/jour)': 'dispo_mat_gr', 'Disponibilité de protéines en quantité (g/personne/jour)': 'dispo_prot'}, inplace=True)

# je remplie les valeurs vide par des valeurs '0'
dispo_alim.fillna(0, inplace=True)

# je convertie cette dataframe en un fichier .csv
dispo_alim.to_csv("dispo_alim.csv", index = False)

### Importer la table 'equilibre_prod'

In [43]:
# j'organise ma dataframe pour pouvoir la telecharger

# je joint les df_animal et df_vegetaux pour avoir mes données
equilibre_prod = ani_veg_q3.copy()

# je rajoute la colonne 'année' contenent la valeur 2013
equilibre_prod['année'] = 2013

# je selectionne les colonnes demandés
equilibre_prod = equilibre_prod[['pays', 'code_zone', 'année', 'produit', 'code_produit', 
                                 'Disponibilité intérieure', 'Aliments pour animaux', 'Semences', 
                                 'Pertes', 'Traitement', 'Nourriture',  'Autres utilisations (non alimentaire)']]

# je renomme mes colonnes pour qu'elles correspondent aux intitulés de la question
equilibre_prod.rename(columns={'code_zone': 'code_pays', 'Disponibilité intérieure': 'dispo_int', 'Aliments pour animaux': 'alim_ani', 'Semences': 'semences', 'Pertes': 'pertes', 'Traitement': 'transfo', 'Nourriture': 'nourriture', 'Autres utilisations (non alimentaire)': 'autres_utilisations'}, inplace=True)

# je remplie les valeurs vide par des valeurs '0'
equilibre_prod.fillna(0, inplace=True)

# je convertie cette dataframe en un fichier .csv
equilibre_prod.to_csv("equilibre_prod.csv", index = False)

### Importer la table 'sous_nutrition'

In [44]:
# j'organise ma dataframe pour pouvoir la telecharger

# je joint les df_animal et df_vegetaux pour avoir mes données
sous_nutrition = df_sousalimentation.copy()

# je selectionne les colonnes demandés
sous_nutrition = sous_nutrition[['pays', 'code_zone', 'année', 'population']]

# je selectionne uniquement les lignes concernant l'année 2013
sous_nutrition = sous_nutrition[sous_nutrition.année == '2012-2014']

# je change le format de mes valeurs pour pouvoir les convertir pour mes calculs plus tard
sous_nutrition['population'] = pd.to_numeric(sous_nutrition['population'], errors='coerce')

# je convertie ma colonne 'population'
sous_nutrition['population'] = sous_nutrition['population'] * 1000000

# je renomme mes colonnes pour qu'elles correspondent aux intitulés de la question
sous_nutrition.rename(columns={'code_zone': 'code_pays', 'population': 'nb_personnes'}, inplace=True)

# je remplie les valeurs vide par des valeurs '0'
sous_nutrition.fillna(0, inplace=True)

# je convertie cette dataframe en un fichier .csv
sous_nutrition.to_csv("sous_nutrition.csv", index = False)